# Imports

In [ ]:
import numpy as np
import pandas as pd
import re
from string import punctuation
import emoji

In [ ]:
import nltk
from nltk.tokenize import word_tokenize, TweetTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN, GRU

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
nltk.download("punkt_tab")

# Vars definition

In [ ]:
n_gram = 3
window_size = 3

# Data preparation

## Initial clean

In [ ]:
df = pd.read_csv("data/data.csv")
df

In [ ]:
df_review = df[["review"]]
df_review

In [ ]:
def clean_text_dataframe(
    df_i: pd.DataFrame,
    columns: list[str] | None = None,
    keep_apostrophe: bool = True,
    min_words: int = 3,
) -> pd.DataFrame:
    """
    Очищает текст и удаляет строки с малым количеством слов

    Params:
        df (pd.DataFrame): Исходный DataFrame
        columns (list[str]|None): Столбцы для обработки (None = все строковые)
        keep_apostrophe (bool): Сохранять апострофы (по умолчанию True)
        min_words (int): Минимальное количество слов для сохранения строки

    Return:
        pd.DataFrame: Очищенная и отфильтрованная копия DataFrame
    """
    df_clean = df_i.copy()

    # Определение целевых столбцов
    if columns is None:
        columns = df_clean.select_dtypes(include=["object", "string"]).columns.tolist()

    # Настройка паттерна для пунктуации
    punct_pattern = r"[{}]".format(
        re.escape(
            punctuation.replace("'", "") if keep_apostrophe else re.escape(punctuation)
        )
    )

    def text_cleaner(text):
        if not isinstance(text, str):
            return text

        # Удаление эмодзи
        text = emoji.replace_emoji(text, replace="")

        # Удаление пунктуации
        text = re.sub(punct_pattern, " ", text)

        # Удаление спецсимволов
        text = re.sub(r"[^a-zA-Z0-9\'\s]", " ", text)

        # Нормализация пробелов
        text = re.sub(r"\s+", " ", text).strip()

        return text

    for col in columns:
        if col in df_clean.columns:
            df_clean[col] = df_clean[col].apply(text_cleaner)

    word_count_mask = (
        df_clean[columns]
        .apply(lambda col: col.str.split().str.len() > min_words)
        .all(axis=1)
    )

    df_clean = df_clean[word_count_mask].reset_index(drop=True)

    return df_clean

In [ ]:
df_review = clean_text_dataframe(df_review, min_words=n_gram)
df_review

## Data tokenize

In [ ]:
def tokenize_text_dataframe(df_i: pd.DataFrame, tokenizer):
    return pd.DataFrame(df_i.iloc[:, 0].apply(lambda col: tokenizer(col.lower())))

In [ ]:
df_tokens = tokenize_text_dataframe(
    df_review, TweetTokenizer(match_phone_numbers=False).tokenize
)
df_tokens

In [ ]:
def vocab_text_dataframe(df_i: pd.DataFrame):
    return pd.DataFrame(df_i.iloc[:, 0].apply(lambda col: sorted(set(col))))


def idx_text_dataframe(df_i: pd.DataFrame):
    return pd.DataFrame(
        df_i.iloc[:, 0].apply(lambda col: {word: idx for idx, word in enumerate(col)})
    )


def global_idx_text_dataframe(df_i: pd.DataFrame):
    """
    Создаёт словарь {слово: индекс} для всех уникальных слов
    из объединённой первой колонки DataFrame, сохраняя порядок появления слов.
    Возвращает DataFrame с одним словарём в виде строки.
    """
    # Объединяем все элементы из первой колонки в один список
    all_words = sum(df_i.iloc[:, 0].tolist(), [])

    # Удаляем дубликаты с сохранением порядка первого появления
    unique_words = list(set(all_words))

    # Создаём итоговый словарь {слово: индекс}
    combined_dict = {word: idx for idx, word in enumerate(unique_words)}

    return pd.DataFrame(list(combined_dict.items()), columns=["Word", "Index"])

In [ ]:
df_vocab = vocab_text_dataframe(df_tokens)
df_word_to_idx = idx_text_dataframe(df_vocab)
df_global_word_to_idx = global_idx_text_dataframe(df_vocab)

In [ ]:
df_vocab

In [ ]:
df_word_to_idx

In [ ]:
df_global_word_to_idx

In [ ]:
tokens = df_tokens.iloc[:, 0].to_list()
vocab = df_vocab.iloc[:, 0].to_list()
global_vocab = list(sorted(set([item for sublist in vocab for item in sublist])))
vocab_size = len(global_vocab)
word_to_idx = df_word_to_idx.iloc[:, 0].to_list()
global_word_to_idx = df_global_word_to_idx.iloc[:, 0].to_list()

In [ ]:
word_to_idx

In [ ]:
global_word_to_idx

In [ ]:
vocab

In [ ]:
global_vocab

## Token preparation

### BoW

In [ ]:
corpus, y_bow = [], []
for idx, cur_token in enumerate(tokens
                                [: 2 * len(tokens) // 3]
                                ):
    for i in range(len(cur_token) - window_size):
        context = cur_token[i : i + window_size]
        corpus.append(" ".join(context))
        y_bow.append(word_to_idx[idx][cur_token[i + window_size]])

vectorizer = CountVectorizer(vocabulary=global_vocab)
X_bow = vectorizer.fit_transform(corpus).toarray()
y_bow = np.array(y_bow)

#### df

In [ ]:
# df_bow = pd.DataFrame({"x": X_bow.tolist(), "y": y_bow.tolist()})
# df_bow

### N-gram

In [ ]:
mass_sequences = []
for idx, cur_token in enumerate(tokens):
    mass_sequences.append([])
    for i in range(len(cur_token) - n_gram + 1):
        mass_sequences[idx].append(cur_token[i : i + n_gram])

X_ngram, y_ngram = [], []
for idx, sequences in enumerate(mass_sequences):
    for seq in sequences:
        # print(seq, word_to_idx)
        X_ngram.append([word_to_idx[idx][word] for word in seq[:-1]])
        y_ngram.append(word_to_idx[idx][seq[-1]])

X_ngram = np.array(X_ngram)
y_ngram = np.array(y_ngram)

#### df

In [ ]:
# df_ngram = pd.DataFrame({"x": X_ngram.tolist(), "y": y_ngram.tolist()})
# df_ngram

## Data split

In [ ]:
X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(
    X_bow, y_bow, test_size=0.2
)

In [ ]:
X_train_ng, X_test_ng, y_train_ng, y_test_ng = train_test_split(
    X_ngram, y_ngram, test_size=0.2
)

# Models

## Graph

In [ ]:
def plot_results(history, title):
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(history.history["accuracy"], label="Train Accuracy")
    plt.plot(history.history["val_accuracy"], label="Test Accuracy")
    plt.title(f"{title} - Accuracy")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history["loss"], label="Train Loss")
    plt.plot(history.history["val_loss"], label="Test Loss")
    plt.title(f"{title} - Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()

    plt.show()

## Model training

### BoW

In [ ]:
model_bow = Sequential(
    [
        Dense(128, activation="relu", input_shape=(vocab_size,)),
        Dense(vocab_size, activation="softmax"),
    ]
)
model_bow.compile(
    loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
)
bow_hist = model_bow.fit(
    X_train_bow, y_train_bow, epochs=10, validation_data=(X_test_bow, y_test_bow)
)

In [ ]:
plot_results(bow_hist, "bow")

In [ ]:
model_bow.save("models/model_bow.keras")

### RNN

In [ ]:
# Модель RNN
model_rnn = Sequential(
    [
        Embedding(vocab_size, 64, input_length=n_gram - 1),
        SimpleRNN(128),
        Dense(vocab_size, activation="softmax"),
    ]
)
model_rnn.compile(
    loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
)
rnn_hist = model_rnn.fit(
    X_train_ng, y_train_ng, epochs=20, validation_data=(X_test_ng, y_test_ng)
)

In [ ]:
plot_results(rnn_hist, "rnn")

In [ ]:
model_rnn.save("models/model_rnn.keras")

### GRU

In [ ]:
# Модель GRU
model_gru = Sequential(
    [
        Embedding(vocab_size, 64, input_length=n_gram - 1),
        GRU(128),
        Dense(vocab_size, activation="softmax"),
    ]
)
model_gru.compile(
    loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
)
gru_hist = model_gru.fit(
    X_train_ng, y_train_ng, epochs=20, validation_data=(X_test_ng, y_test_ng)
)

In [ ]:
plot_results(gru_hist, "gru")

In [ ]:
model_gru.save("models/model_gru.keras")

In [ ]:
# Оценка моделей
def evaluate_model(model, X_test, y_test, name):
    y_pred = model.predict(X_test).argmax(axis=1)
    print(f"\n{name} Classification Report:")
    print(classification_report(y_test, y_pred, zero_division=0))

In [ ]:
evaluate_model(model_bow, X_test_bow, y_test_bow, "BoW")
evaluate_model(model_rnn, X_test_ng, y_test_ng, "RNN")
evaluate_model(model_gru, X_test_ng, y_test_ng, "GRU")

# Word prediction

In [ ]:
def predict_next_word(
    model, input_sequence, word_to_idx, idx_to_word, mode="ngram", top_k=3
):
    """
    Предсказывает следующее слово на основе входной последовательности.

    Параметры:
        model: обученная модель (Keras или sklearn).
        input_sequence: исходное предложение (строка).
        word_to_idx: словарь для преобразования слов в индексы.
        idx_to_word: словарь для преобразования индексов в слова.
        mode: тип модели ("ngram" или "bow").
        top_k: количество вариантов для вывода.
    """
    # Токенизация и преобразование в нижний регистр
    tokens = word_tokenize(input_sequence.lower())
    tokens_idx = [
        word_to_idx.get(word, -1) for word in tokens
    ]  # -1 для неизвестных слов

    # Обработка неизвестных слов (замена на <UNK> или пропуск)
    tokens_idx = [
        idx if idx != -1 else word_to_idx.get("<UNK>", -1) for idx in tokens_idx
    ]
    if -1 in tokens_idx:
        print("Есть неизвестные слова!")
        return []

    # Подготовка данных в зависимости от типа модели
    if mode == "bow":
        # Используем последние window_size слов как контекст
        window_size = 5  # Должно совпадать с обучением!
        context = tokens_idx[-window_size:]
        if len(context) < window_size:
            # Дополняем нулями слева (pad_sequences)
            context = [0] * (window_size - len(context)) + context

        # Создаем вектор BoW (количество вхождений каждого слова)
        bow_vector = np.zeros(len(word_to_idx))
        for idx in context:
            if idx < len(word_to_idx):
                bow_vector[idx] += 1
        input_data = bow_vector.reshape(1, -1)

    elif mode == "ngram":
        # Используем последние n-1 слов для N-граммной модели
        n_gram = 3  # Должно совпадать с обучением!
        seq_length = n_gram - 1
        context = tokens_idx[-seq_length:]
        if len(context) < seq_length:
            # Дополняем нулями слева
            context = [0] * (seq_length - len(context)) + context

        input_data = np.array([context])

    else:
        raise ValueError("Режим должен быть 'bow' или 'ngram'")

    # Предсказание
    preds = model.predict(input_data)[0]
    top_indices = preds.argsort()[-top_k:][::-1]  # Топ-K индексов
    top_words = [idx_to_word[idx] for idx in top_indices if idx in idx_to_word]

    return top_words

In [ ]:
input_sentence = "I love"
idx_to_word = {v: k for k, v in word_to_idx[0].items()}  # Создаем обратный словарь

# Предсказание через BoW
bow_prediction = predict_next_word(
    model_bow, input_sentence, word_to_idx[0], idx_to_word, mode="bow", top_k=3
)
" ".join([input_sentence, bow_prediction[0]])

In [ ]:
input_sentence = "I love"
idx_to_word = {v: k for k, v in word_to_idx.items()}  # Создаем обратный словарь

# Предсказание через BoW
bow_prediction = predict_next_word(
    model_rnn, input_sentence, word_to_idx, idx_to_word, mode="ngram", top_k=3
)
" ".join([input_sentence, bow_prediction[0]])